In [1]:
from peakaboo.peak_character import find_nearest
#from peakaboo.peak_character import peakchar
from peakaboo.peak_character import convert_to_nm
from peakaboo.peak_character import peak_matrix
import numpy as np
import pandas as pd

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

In [ ]:
def test_find_nearest():
    array = 'string'
    value = 1
    try:
        find_nearest(array, value)
    except TypeError:
        pass
    else:
        print('TypeError not handled')

In [ ]:
test_find_nearest()

In [2]:
def peakchar(data_nm, data_z_array, peak_index):
    """
    find the peak height and width
    
    Args:
        data_nm: wavelength array, numpy array
        data_z_array: data array
        peak_index: index of the peaks identified
    
    Returns:
        height: peak intensity, numpy array
        fwhm: widths pf peaks defined as full-width half-max, numpy array
        
    """
    num_peaks = len(peak_index)
    
    #array of peak height
    height = [data_z_array[idx] for idx in peak_index]
    
    #array of peak width
    half_height = [ht / 2 for ht in height]

    fwhm_idx_1 = np.empty_like(half_height)
    fwhm_idx_2 = np.empty_like(fwhm_idx_1)
    fwhm_nm_1 = np.empty_like(fwhm_idx_1)
    fwhm_nm_2 = np.empty_like(fwhm_idx_1)
    
    for i in range(num_peaks):
        #find the index and nm corresponding to half of the peak intensity
        #on the left side of the peak
        if i == 0:
            fwhm_idx_1[i] = find_nearest(data_z_array[0:peak_index[i]], half_height[i])
        else:
            fwhm_idx_1[i] = find_nearest(data_z_array[peak_index[i-1]:peak_index[i]], half_height[i]) + peak_index[i-1]

        print(np.shape(fwhm_idx_1[i]))
        print(fwhm_idx_1)
        fwhm_nm_1[i] = data_nm[int(fwhm_idx_1[i])]
        
        #find the index and nm corresponding to half of the peak intensity
        #on the right side of the peak   
        fwhm_idx_2[i] = find_nearest(data_z_array[peak_index[i]:], half_height[i]) + peak_index[i]
        fwhm_nm_2[i] = data_nm[int(fwhm_idx_2[i])]
    
    #calculate fwhm as the difference between the index/nm's on the left and
    #right side of the peak
    fwhm = fwhm_nm_2 - fwhm_nm_1

    return height, fwhm

In [3]:
def test_peakchar():
    data_nm = 'string'
    data_z = np.zeros((5, 5))
    peak_idx = pd.DataFrame([[1, 2, 3], [1, 2, 3], \
                             [1, 2, 3], [1, 2, 3], \
                             [1, 2,3]])
    try:
        peakchar(data_nm, data_z, peak_idx)
    except TypeError:
        pass
    else:
        print('TypeError not handled', 'Check peak_finding output')
        
    data_nm = np.random.rand(144)
    data_z = np.random.rand(144, 707)
    peak_idx = np.array([1, 2, 3])
    height, fwhm = peakchar(data_nm, data_z, peak_idx)
    if isinstance(height, np.array):
        pass
    else:
        raise Exception('Bad type', 'Height is not np array')
        
    if isinstance(fwhm, np.array):
        pass
    else:
        raise Exception('Bad type', 'Fwhm is not np array')

In [4]:
test_peakchar()

(707,)
[[505. 505. 505. ... 505. 505. 505.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


TypeError: only size-1 arrays can be converted to Python scalars

In [ ]:
peak_idx = [1, 2, 3]
peak_idx.dtype